# Iterative Methods for Solving Linear Systems

We are solving the following system of linear equations:

\begin{aligned}
5x_1 - 2x_2 + 3x_3 &= -1 \\
-3x_1 + 9x_2 + x_3 &= 2 \\
2x_1 - x_2 - 7x_3 &= 3
\end{aligned}

The basic procedure is to start with an initial guess for the solution and repeatedly refine it by using the given equations to find the next values of the variables. The process is repeated until the solution is accurate enough.

## Initial Guess

We start with the initial guess:$\quad x_1^{(0)} = 0,\quad x_2^{(0)} = 0,\quad x_3^{(0)} = 0$

---

## Gauss-Jacobi Method

In the Jacobi method, we compute each new value using only the values from the **previous iteration**.

Update formulas:

\begin{aligned}
x_1^{(k+1)} &= \frac{1}{5} \left( -1 + 2x_2^{(k)} - 3x_3^{(k)} \right) \\
x_2^{(k+1)} &= \frac{1}{9} \left( 2 + 3x_1^{(k)} - x_3^{(k)} \right) \\
x_3^{(k+1)} &= \frac{1}{-7} \left( 3 - 2x_1^{(k)} + x_2^{(k)} \right)
\end{aligned}

## Gauss-Seidel Method

In the Gauss-Seidel method, we update values **in-place**, using the latest computed values within the same iteration.

Update formulas:
\begin{aligned}
x_1^{(k+1)} &= \frac{1}{5} \left( -1 + 2x_2^{(k)} - 3x_3^{(k)} \right) \\
x_2^{(k+1)} &= \frac{1}{9} \left( 2 + 3x_1^{(k+1)} - x_3^{(k)} \right) \\
x_3^{(k+1)} &= \frac{1}{-7} \left( 3 - 2x_1^{(k+1)} + x_2^{(k+1)} \right)
\end{aligned}

## Successive Over-Relaxation (SOR) Method

SOR is a modification of Gauss-Seidel that introduces a **relaxation parameter** $\omega$, $1 < \omega < 2$. The goal is to accelerate convergence.

Update formulas:

\begin{aligned}
x_1^{(k+1)} &= (1 - \omega)x_1^{(k)} + \frac{\omega}{5} \left( -1 + 2x_2^{(k)} - 3x_3^{(k)} \right) \\
x_2^{(k+1)} &= (1 - \omega)x_2^{(k)} + \frac{\omega}{9} \left( 2 + 3x_1^{(k+1)} - x_3^{(k)} \right) \\
x_3^{(k+1)} &= (1 - \omega)x_3^{(k)} + \frac{\omega}{-7} \left( 3 - 2x_1^{(k+1)} + x_2^{(k+1)} \right)
\end{aligned}

---

## Notes

- Gauss-Seidel generally converges faster than Gauss-Jacobi.
- SOR can outperform both if $\omega$ is chosen well.
- All methods assume the matrix is diagonally dominant or suitable for convergence.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

'''
Solving the system of equations:
5x - 2y +3z = -1
-3x + 9y + z = 2
2x - y - 7z = 3
'''

# Define coefficient matrix and RHS
A = np.array([
    [5, -2, 3],
    [-3, 9, 1],
    [2, -1, -7]
], dtype=float)

b = np.array([-1, 2, 3], dtype=float)

# Define initial value
x_init = np.array([0, 0, 0], dtype=float)         # initial guess x_0 = [0,0,0]

In [9]:
def gauss_jacobi(x_init, A, b, iterations=10):
    n = len(b)
    x = x_init
    history = []

    for _ in range(iterations):
        x_new = np.zeros(n)
        for i in range(n):
            s = sum(A[i][j] * x[j] for j in range(n) if j != i)
            x_new[i] = (b[i] - s) / A[i][i]
        diff = np.abs(x_new - x)
        history.append(np.concatenate((x_new, diff)))
        x = x_new
    cols = ["x1 (J)", "x2 (J)", "x3 (J)", "Δx1 (J)", "Δx2 (J)", "Δx3 (J)"]
    return pd.DataFrame(history, columns=cols)

def gauss_seidel(x_init, A, b, iterations=10):
    n = len(b)
    x = x_init
    history = []

    for _ in range(iterations):
        x_new = x.copy()
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i][i]
        diff = np.abs(x_new - x)
        history.append(np.concatenate((x_new, diff)))
        x = x_new
    cols = ["x1 (S)", "x2 (S)", "x3 (S)", "Δx1 (S)", "Δx2 (S)", "Δx3 (S)"]
    return pd.DataFrame(history, columns=cols)

def sor_method(x_init, A, b, w=1.25, iterations=10):     # default w = 1.25
    n = len(b)
    x = x_init
    history = []

    for _ in range(iterations):
        x_new = x.copy()
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))       # updated
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))     # previous
            x_new[i] = (1 - w) * x[i] + w * (b[i] - s1 - s2) / A[i][i]
        diff = np.abs(x_new - x)
        history.append(np.concatenate((x_new, diff)))
        x = x_new
    cols = ["x1 (SOR)", "x2 (SOR)", "x3 (SOR)", "Δx1 (SOR)", "Δx2 (SOR)", "Δx3 (SOR)"]
    return pd.DataFrame(history, columns=cols)

In [10]:
jacobi_df = gauss_jacobi(x_init, A, b)
print("Result of Gauss-Jacobi method: ")
print(jacobi_df)

seidel_df = gauss_seidel(x_init, A, b)
print("\nResult of Gauss-Seidel method: ")
print(seidel_df)

sor_df = sor_method(x_init, A, b)
print("\nResult of SOR method: ")
print(sor_df)

# True solution for reference
x_true = np.linalg.solve(A, b)
print("\nTrue solution:\n", x_true)

Result of Gauss-Jacobi method: 
     x1 (J)    x2 (J)    x3 (J)   Δx1 (J)   Δx2 (J)   Δx3 (J)
0 -0.200000  0.222222 -0.428571  0.200000  0.222222  0.428571
1  0.146032  0.203175 -0.517460  0.346032  0.019048  0.088889
2  0.191746  0.328395 -0.415873  0.045714  0.125220  0.101587
3  0.180882  0.332346 -0.420700  0.010864  0.003951  0.004827
4  0.185359  0.329261 -0.424369  0.004477  0.003085  0.003668
5  0.186326  0.331160 -0.422649  0.000967  0.001900  0.001720
6  0.186054  0.331292 -0.422644  0.000272  0.000131  0.000005
7  0.186103  0.331201 -0.422741  0.000050  0.000091  0.000096
8  0.186125  0.331228 -0.422713  0.000021  0.000027  0.000027
9  0.186119  0.331232 -0.422711  0.000005  0.000004  0.000002

Result of Gauss-Seidel method: 
     x1 (S)    x2 (S)    x3 (S)       Δx1 (S)       Δx2 (S)       Δx3 (S)
0 -0.200000  0.155556 -0.507937  2.000000e-01  1.555556e-01  5.079365e-01
1  0.166984  0.334321 -0.428622  3.669841e-01  1.787654e-01  7.931469e-02
2  0.190901  0.333481 -0.421668

### Comparison of Results
As can be seen, both methods give good results after 10 iterations. However, there are some key differences:  
- The Gauss-Seidel method converges faster (almost 2X) than the Jacobi method. This is because the Gauss-Seidel method uses the latest values of the variables immediately, whereas the Jacobi method uses the old values.
- The Gauss-Jacobi method has the advantage of being highly parallelizable.
- The SOR method is faster than both the Gauss-Seidel and Jacobi methods. However, it requires a parameter ($\omega$) that needs to be chosen carefully. If $\omega$ is too small, the method converges slowly, and if it is too large, the method may not converge at all. Finding the best fit $\omega$ is not an easy task.